### Setup

In [1]:
// import dependencies

var moment = require('moment')
var { constants } = require('../utils')
var { advanceTimeAndBlock } = require('../utils/evm')
var { evm, common, contracts } = require('../utils');
var { toUnit } = require('../utils/bn')
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm')
var { next, clear, bnToNumber } = require('../utils/jupyter')
var { Notebook } = require('../utils/notebook')
var { ethers } = require('hardhat')
var { BigNumber } = require('ethers')
var { smock } = require('@defi-wonderland/smock')

var $ = new Notebook();

clear();

In [2]:
// setup tools and contracts

next(async () => {
    await $.setup(constants.FORK_BLOCK_NUMBER);
    
    provider = await $.impersonate(constants.RICH_DAI_ADDRESS);
    
    // contract artifacts
    ls = await $.fetch('LongShort', constants.LONG_SHORT_ADDRESS)
    synthF3Long = await $.fetch('SyntheticToken', constants.SYNTHETIC_TOKEN_ADDRESS_F3_LONG)
    synthF3Short = await $.fetch('SyntheticToken',constants.SYNTHETIC_TOKEN_ADDRESS_F3_SHORT)
    ym = await $.fetch('YieldManagerAave', constants.YIELD_MANAGER_ADDRESS)
    oracleManager = await $.fetch('OracleManagerFlippening_V0', constants.ORACLE_MANAGER_ADDRESS_F3)
    alphaFLT = await $.fetch('AlphaTestFLT',constants.ALPHA_FLT_ADDRESS)
    treasury = await $.fetch('TreasuryAlpha',constants.TREASURY_ALPHA_ADDRESS)
    
    dai = await $.fetch('ERC20', constants.DAI_ADDRESS )
    
    // fake contracts
    oracleManagerFake = await smock.fake('OracleManagerFlippening_V0');
});

In [3]:
//Updating the address of the oracleManager manager to the fake oracleManager contract
next(async()=>{
    // initialize a value so it doesn't return 0
    ORACLE_BASE = await oracleManager.getLatestPrice()
    oracleManagerFake.updatePrice.returns(ORACLE_BASE)

    ls.connect(await $.impersonate('0xcfcCB5e6b3882b7795eb1B1A9f18831A842d4eE0')).updateMarketOracle(1,oracleManagerFake.address)    
})

In [4]:
//Mint long next price for user in F3 market
next(async()=>{
    DEPOSIT = '1000000000000000000000000' // 100K DAI
    
    await dai.connect(provider).approve(ls.address, DEPOSIT)
    await ls.connect(provider).mintLongNextPrice(await synthF3Long.marketIndex(), DEPOSIT)
})

In [5]:
// charts configuration

next(async() =>{
    await $.resetRecording()
    $.resetTraces()

    $.setPeriodTrace(432000)
    
    $.addViewTrace(synthF3Long, 'totalSupply', [], 'synthF3Long totalSupply')
    $.addViewTrace(alphaFLT, 'totalSupply', [], 'alphaFLT totalSupply')
    $.addViewTrace(ls, 'marketSideValueInPaymentToken', [1,true], 'marketSideValueInPaymentToken')
})

In [6]:
//Trigger a system state update
next(async()=>{
    await $.recordViews()
    //Faking a change in the oracle return
    oracleManagerFake.updatePrice.returns(ORACLE_BASE + 10000002)
    //Forcing an oracle price update for next price execution to go through
    await ls.connect(provider).updateSystemState(1)
    
    //Running next price execution for the user
    await ls.executeOutstandingNextPriceSettlementsUser(constants.RICH_DAI_ADDRESS, 1)
    
    await $.recordViews()
    
    await $.draw()
})